### XGBoost + oversampling & undersampling SMOTE

The team saw the potential model performance from Oversampling and Undersampling SMOTE. In this standalone notebook, the team explore the potential performance boost on XGBoost classifier. Similar to the sample treatment on Random Forest Classifier, the team applied oversampling and undersampling SMOTE. First oversample the minority class with SMOTE to about a 1:10 ratio, then undersample the majority class to achieve about a 1:2 ratio.

In [1]:
# Import libraries 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

### Import files

In [2]:
# Import the files
df_test = pd.read_csv("https://raw.githubusercontent.com/AngShengJun/dsi14P4/master/assets/working/df_test_weather_cleaned.csv")
df_train = pd.read_csv("https://raw.githubusercontent.com/AngShengJun/dsi14P4/master/assets/working/df_train_weather_cleaned.csv")

In [3]:
df_test.head(1)

,Unnamed: 0,id,date,lat,long,wk,yr,tavg,stnpress,dewpt,...,trap_T230,trap_T231,trap_T232,trap_T233,trap_T235,trap_T236,trap_T237,trap_T238,trap_T900,trap_T903
0,0,1,2008-06-11,41.95469,-87.800991,23,2008,75.0,29.31,55.5,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df_train.head(1)

,Unnamed: 0,date,lat,long,wnv,num_mos,wk,yr,tavg,stnpress,...,trap_T230,trap_T231,trap_T232,trap_T233,trap_T235,trap_T236,trap_T237,trap_T238,trap_T900,trap_T903
0,0,2007-05-29,41.95469,-87.800991,0,1,22,2007,75.5,29.415,...,0,0,0,0,0,0,0,0,0,0


### Prep

In [5]:
# Make a copy of test
df_testcopy = df_test.copy()

In [6]:
# Drop unnecessary col
df_testcopy.drop(['id','Unnamed: 0'],axis=1,inplace=True)
# Review
df_testcopy.head(1)

,date,lat,long,wk,yr,tavg,stnpress,dewpt,precip,windspeed,...,trap_T230,trap_T231,trap_T232,trap_T233,trap_T235,trap_T236,trap_T237,trap_T238,trap_T900,trap_T903
0,2008-06-11,41.95469,-87.800991,23,2008,75.0,29.31,55.5,0.0,9.15,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Make a copy of train
df_traincopy = df_train.copy()

In [8]:
# Drop unnecessary col
df_traincopy.drop(['Unnamed: 0'],axis=1,inplace=True)
# Review
df_traincopy.head(1)

,date,lat,long,wnv,num_mos,wk,yr,tavg,stnpress,dewpt,...,trap_T230,trap_T231,trap_T232,trap_T233,trap_T235,trap_T236,trap_T237,trap_T238,trap_T900,trap_T903
0,2007-05-29,41.95469,-87.800991,0,1,22,2007,75.5,29.415,58.5,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Sanity check
print(df_traincopy.shape)
print(df_testcopy.shape)

(8610, 156)
(116293, 154)


In [10]:
# Drop date column from test and train
df_traincopy.drop(['date'],axis=1,inplace=True)
df_testcopy.drop(['date'],axis=1,inplace=True)

In [11]:
# trc - TrainComplete set
X_trc = df_traincopy.drop(['num_mos','wnv'],axis=1)
y_trc = df_traincopy['wnv']

In [12]:
df_traincopy.head(1)

,lat,long,wnv,num_mos,wk,yr,tavg,stnpress,dewpt,precip,...,trap_T230,trap_T231,trap_T232,trap_T233,trap_T235,trap_T236,trap_T237,trap_T238,trap_T900,trap_T903
0,41.95469,-87.800991,0,1,22,2007,75.5,29.415,58.5,0.0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df_testcopy.head(1)

,lat,long,wk,yr,tavg,stnpress,dewpt,precip,windspeed,daylight,...,trap_T230,trap_T231,trap_T232,trap_T233,trap_T235,trap_T236,trap_T237,trap_T238,trap_T900,trap_T903
0,41.95469,-87.800991,23,2008,75.0,29.31,55.5,0.0,9.15,15.166667,...,0,0,0,0,0,0,0,0,0,0


### Train-Validate-Split

In [14]:
# Train-validate-split
X_train,X_val,y_train,y_val = train_test_split(X_trc,y_trc,test_size=0.3,random_state=42, stratify=y_trc)

In [15]:
print(y_train.value_counts(normalize=True))

0    0.946906
1    0.053094
Name: wnv, dtype: float64


### Train-Validate Evaluation

In [16]:
from imblearn.over_sampling import SMOTE


# define pipeline



Using TensorFlow backend.


In [17]:
from imblearn.under_sampling import RandomUnderSampler



In [18]:
from imblearn.pipeline import Pipeline

In [19]:
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier

In [20]:
xg_pipeline = Pipeline([
    ('over', SMOTE(random_state=42)),
    ('under', RandomUnderSampler(random_state=42)),
    ('xgb', XGBClassifier(random_state=42))
])

In [21]:
# Define dictionary of hyperparameters.
pipeline_params = {
    'over__k_neighbors' : [1,2,3,4,5,6,7,8,9,10],
    'over__sampling_strategy' : [0.1],
    'under__sampling_strategy' : [0.5],
    'xgb__max_depth': [9,11],
    'xgb__learning_rate' : [0.1],
    'xgb__n_estimators' : [100,125],
    'xgb__objective' : ['binary:logistic'],
    'xgb__gamma': [0.5,1],
    'xgb__min_child_weight': [1,5],
    'xgb__subsample': [0.5,1.0],
    'xgb__colsample_bytree': [0.5,1.0]
}

In [22]:
# Instantiate our GridSearchCV object.
xg_gs = GridSearchCV(xg_pipeline,
                     pipeline_params,
                     cv=5,
                     verbose=2,
                     scoring='roc_auc')

In [23]:
# Fit the GridSearchCV object to the data.
xg_gs.fit(X_train, y_train)

Fitting 5 folds for each of 640 candidates, totalling 3200 fits
[CV] over__k_neighbors=1, over__sampling_strategy=0.1, under__sampling_strategy=0.5, xgb__colsample_bytree=0.5, xgb__gamma=0.5, xgb__learning_rate=0.1, xgb__max_depth=9, xgb__min_child_weight=1, xgb__n_estimators=100, xgb__objective=binary:logistic, xgb__subsample=0.5 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  over__k_neighbors=1, over__sampling_strategy=0.1, under__sampling_strategy=0.5, xgb__colsample_bytree=0.5, xgb__gamma=0.5, xgb__learning_rate=0.1, xgb__max_depth=9, xgb__min_child_weight=1, xgb__n_estimators=100, xgb__objective=binary:logistic, xgb__subsample=0.5, total=   0.7s
[CV] over__k_neighbors=1, over__sampling_strategy=0.1, under__sampling_strategy=0.5, xgb__colsample_bytree=0.5, xgb__gamma=0.5, xgb__learning_rate=0.1, xgb__max_depth=9, xgb__min_child_weight=1, xgb__n_estimators=100, xgb__objective=binary:logistic, xgb__subsample=0.5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV]  over__k_neighbors=1, over__sampling_strategy=0.1, under__sampling_strategy=0.5, xgb__colsample_bytree=0.5, xgb__gamma=0.5, xgb__learning_rate=0.1, xgb__max_depth=9, xgb__min_child_weight=1, xgb__n_estimators=100, xgb__objective=binary:logistic, xgb__subsample=0.5, total=   0.8s
[CV] over__k_neighbors=1, over__sampling_strategy=0.1, under__sampling_strategy=0.5, xgb__colsample_bytree=0.5, xgb__gamma=0.5, xgb__learning_rate=0.1, xgb__max_depth=9, xgb__min_child_weight=1, xgb__n_estimators=100, xgb__objective=binary:logistic, xgb__subsample=0.5 
[CV]  over__k_neighbors=1, over__sampling_strategy=0.1, under__sampling_strategy=0.5, xgb__colsample_bytree=0.5, xgb__gamma=0.5, xgb__learning_rate=0.1, xgb__max_depth=9, xgb__min_child_weight=1, xgb__n_estimators=100, xgb__objective=binary:logistic, xgb__subsample=0.5, total=   0.8s
[CV] over__k_neighbors=1, over__sampling_strategy=0.1, under__sampling_strategy=0.5, xgb__colsample_bytree=0.5, xgb__gamma=0.5, xgb__learning_rate=0.1, xgb__max

...

[CV]  over__k_neighbors=10, over__sampling_strategy=0.1, under__sampling_strategy=0.5, xgb__colsample_bytree=1.0, xgb__gamma=1, xgb__learning_rate=0.1, xgb__max_depth=11, xgb__min_child_weight=1, xgb__n_estimators=100, xgb__objective=binary:logistic, xgb__subsample=1.0, total=   1.6s
[CV] over__k_neighbors=10, over__sampling_strategy=0.1, under__sampling_strategy=0.5, xgb__colsample_bytree=1.0, xgb__gamma=1, xgb__learning_rate=0.1, xgb__max_depth=11, xgb__min_child_weight=1, xgb__n_estimators=100, xgb__objective=binary:logistic, xgb__subsample=1.0 
[CV]  over__k_neighbors=10, over__sampling_strategy=0.1, under__sampling_strategy=0.5, xgb__colsample_bytree=1.0, xgb__gamma=1, xgb__learning_rate=0.1, xgb__max_depth=11, xgb__min_child_weight=1, xgb__n_estimators=100, xgb__objective=binary:logistic, xgb__subsample=1.0, total=   1.6s
[CV] over__k_neighbors=10, over__sampling_strategy=0.1, under__sampling_strategy=0.5, xgb__colsample_bytree=1.0, xgb__gamma=1, xgb__learning_rate=0.1, xgb__max_

[CV]  over__k_neighbors=10, over__sampling_strategy=0.1, under__sampling_strategy=0.5, xgb__colsample_bytree=1.0, xgb__gamma=1, xgb__learning_rate=0.1, xgb__max_depth=11, xgb__min_child_weight=5, xgb__n_estimators=100, xgb__objective=binary:logistic, xgb__subsample=0.5, total=   0.9s
[CV] over__k_neighbors=10, over__sampling_strategy=0.1, under__sampling_strategy=0.5, xgb__colsample_bytree=1.0, xgb__gamma=1, xgb__learning_rate=0.1, xgb__max_depth=11, xgb__min_child_weight=5, xgb__n_estimators=100, xgb__objective=binary:logistic, xgb__subsample=0.5 
[CV]  over__k_neighbors=10, over__sampling_strategy=0.1, under__sampling_strategy=0.5, xgb__colsample_bytree=1.0, xgb__gamma=1, xgb__learning_rate=0.1, xgb__max_depth=11, xgb__min_child_weight=5, xgb__n_estimators=100, xgb__objective=binary:logistic, xgb__subsample=0.5, total=   0.9s
[CV] over__k_neighbors=10, over__sampling_strategy=0.1, under__sampling_strategy=0.5, xgb__colsample_bytree=1.0, xgb__gamma=1, xgb__learning_rate=0.1, xgb__max_

[CV]  over__k_neighbors=10, over__sampling_strategy=0.1, under__sampling_strategy=0.5, xgb__colsample_bytree=1.0, xgb__gamma=1, xgb__learning_rate=0.1, xgb__max_depth=11, xgb__min_child_weight=5, xgb__n_estimators=125, xgb__objective=binary:logistic, xgb__subsample=1.0, total=   1.7s
[CV] over__k_neighbors=10, over__sampling_strategy=0.1, under__sampling_strategy=0.5, xgb__colsample_bytree=1.0, xgb__gamma=1, xgb__learning_rate=0.1, xgb__max_depth=11, xgb__min_child_weight=5, xgb__n_estimators=125, xgb__objective=binary:logistic, xgb__subsample=1.0 
[CV]  over__k_neighbors=10, over__sampling_strategy=0.1, under__sampling_strategy=0.5, xgb__colsample_bytree=1.0, xgb__gamma=1, xgb__learning_rate=0.1, xgb__max_depth=11, xgb__min_child_weight=5, xgb__n_estimators=125, xgb__objective=binary:logistic, xgb__subsample=1.0, total=   1.7s
[CV] over__k_neighbors=10, over__sampling_strategy=0.1, under__sampling_strategy=0.5, xgb__colsample_bytree=1.0, xgb__gamma=1, xgb__learning_rate=0.1, xgb__max_

[Parallel(n_jobs=1)]: Done 3200 out of 3200 | elapsed: 60.6min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('over',
                                        SMOTE(k_neighbors=5, n_jobs=None,
                                              random_state=42,
                                              sampling_strategy='auto')),
                                       ('under',
                                        RandomUnderSampler(random_state=42,
                                                           replacement=False,
                                                           sampling_strategy='auto')),
                                       ('xgb',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                             

In [24]:
# Save the best model
xg_gs_best = xg_gs.best_estimator_

In [25]:
# Checking the scores 
print(f"Roc_auc on train set: {xg_gs_best.score(X_train, y_train)}")
print(f"Roc_auc on validate set: {xg_gs_best.score(X_val, y_val)}")

Roc_auc on train set: 0.8616226978596316
Roc_auc on validate set: 0.8416569879984515


In [26]:
prediction_xg = xg_gs_best.predict(X_val)

In [27]:
# Confusion matrix
cm = confusion_matrix(y_val, prediction_xg)
# Save TN/FP/FN/TP values.
tn, fp, fn, tp = confusion_matrix(y_val,prediction_xg).ravel()

In [28]:
# Summary of metrics for random forest model
sens = tp/(tp+fn)
prec = tp/(tp+fp)
f1 = 2*(prec*sens)/(prec+sens)
print(f"Sensitivity: {round(sens,4)}")
print(f"Precision: {round(prec,4)}")
print(f"F1: {round(f1,4)}")

Sensitivity: 0.6569
Precision: 0.1991
F1: 0.3056


In [29]:
pred_proba = [i[1] for i in xg_gs_best.predict_proba(X_val)]

pred_df = pd.DataFrame({'validate_values': y_val,
                        'pred_probs':pred_proba})
pred_df.head()

,validate_values,pred_probs
5602,0,0.603087
1012,0,0.235592
7561,0,0.268146
7887,0,0.372212
4422,0,0.083188


In [30]:
# Calculate ROC AUC.
roc_auc_score(pred_df['validate_values'],pred_df['pred_probs'])

0.8535848786339683

### Best Model 3 (XGBoost+Over & UnderSampling) Evaluation

In [31]:
# Check train set headers
df_traincopy.head(1)

,lat,long,wnv,num_mos,wk,yr,tavg,stnpress,dewpt,precip,...,trap_T230,trap_T231,trap_T232,trap_T233,trap_T235,trap_T236,trap_T237,trap_T238,trap_T900,trap_T903
0,41.95469,-87.800991,0,1,22,2007,75.5,29.415,58.5,0.0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# Check test set headers
df_testcopy.head(1)

,lat,long,wk,yr,tavg,stnpress,dewpt,precip,windspeed,daylight,...,trap_T230,trap_T231,trap_T232,trap_T233,trap_T235,trap_T236,trap_T237,trap_T238,trap_T900,trap_T903
0,41.95469,-87.800991,23,2008,75.0,29.31,55.5,0.0,9.15,15.166667,...,0,0,0,0,0,0,0,0,0,0


In [33]:
# making a copy of the train_kaggle df
X_train_kaggle = df_traincopy.copy().drop(['num_mos','wnv'],axis=1)
y_train_kaggle = df_traincopy.copy()['wnv']

X_test_kaggle = df_testcopy.copy()

In [34]:
X_train_kaggle.columns.difference(X_test_kaggle.columns)

Index([], dtype='object')

In [35]:
X_test_kaggle.columns.difference(X_train_kaggle.columns)

Index([], dtype='object')

In [36]:
#checking shape
X_train_kaggle.shape

(8610, 153)

In [37]:
X_train_kaggle.head()

,lat,long,wk,yr,tavg,stnpress,dewpt,precip,windspeed,daylight,...,trap_T230,trap_T231,trap_T232,trap_T233,trap_T235,trap_T236,trap_T237,trap_T238,trap_T900,trap_T903
0,41.954690,-87.800991,22,2007,75.5,29.415,58.5,0.0,5.8,15.6,...,0,0,0,0,0,0,0,0,0,0
1,41.954690,-87.800991,22,2007,75.5,29.415,58.5,0.0,5.8,15.6,...,0,0,0,0,0,0,0,0,0,0
2,41.994991,-87.769279,22,2007,75.5,29.415,58.5,0.0,5.8,15.6,...,0,0,0,0,0,0,0,0,0,0
3,41.974089,-87.824812,22,2007,75.5,29.415,58.5,0.0,5.8,15.6,...,0,0,0,0,0,0,0,0,0,0
4,41.974089,-87.824812,22,2007,75.5,29.415,58.5,0.0,5.8,15.6,...,0,0,0,0,0,0,0,0,0,0


In [38]:
X_test_kaggle.head()

,lat,long,wk,yr,tavg,stnpress,dewpt,precip,windspeed,daylight,...,trap_T230,trap_T231,trap_T232,trap_T233,trap_T235,trap_T236,trap_T237,trap_T238,trap_T900,trap_T903
0,41.95469,-87.800991,23,2008,75.0,29.31,55.5,0.0,9.15,15.166667,...,0,0,0,0,0,0,0,0,0,0
1,41.95469,-87.800991,23,2008,75.0,29.31,55.5,0.0,9.15,15.166667,...,0,0,0,0,0,0,0,0,0,0
2,41.95469,-87.800991,23,2008,75.0,29.31,55.5,0.0,9.15,15.166667,...,0,0,0,0,0,0,0,0,0,0
3,41.95469,-87.800991,23,2008,75.0,29.31,55.5,0.0,9.15,15.166667,...,0,0,0,0,0,0,0,0,0,0
4,41.95469,-87.800991,23,2008,75.0,29.31,55.5,0.0,9.15,15.166667,...,0,0,0,0,0,0,0,0,0,0


In [39]:
#checking shape
X_test_kaggle.shape

(116293, 153)

In [40]:
# try random forest for kaggle submission
xg_model = xg_gs_best

In [41]:
xg_model.fit(X_train_kaggle, y_train_kaggle)

Pipeline(memory=None,
         steps=[('over',
                 SMOTE(k_neighbors=5, n_jobs=None, random_state=42,
                       sampling_strategy=0.1)),
                ('under',
                 RandomUnderSampler(random_state=42, replacement=False,
                                    sampling_strategy=0.5)),
                ('xgb',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1.0, gamma=1, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='', learning_rate=0.1,
                               max_delta_step=0, max_depth=9,
                               min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=125,
                               n_jobs=0, num_parallel_tree=1,
                               objective='binar

In [42]:
print(f"Accuracy on train set: {xg_model.score(X_train_kaggle,y_train_kaggle)}")

Accuracy on train set: 0.8711962833914053


In [43]:
# predicting kaggle output
predict_kaggle = xg_model.predict(X_test_kaggle)

In [44]:
# probability predicition
predict_proba_kaggle = xg_model.predict_proba(X_test_kaggle)

In [45]:
#Saving an output CSV file for submission
output3 = pd.DataFrame({'Id': df_test['id'], 'WnvPresent': predict_proba_kaggle[:,1]})
output3.to_csv('my_submission_DSI3.csv', index=False)

### Conclusion

In [46]:
# Summary of Model scores in Dataframe
summary_df = pd.DataFrame({'roc_auc(val)':[0.847,0.845],\
                           'sensitivity':[0.664, 0.650],\
                           'precision':[0.212, 0.202],\
                           'F1':[0.322, 0.309],\
                           'roc_auc':[0.866,0.860]})
# Transpose dataframe
summary_dft = summary_df.T
# Rename columns
summary_dft.columns = ['RF(Smote O&U)', 'XGBc(Smote O&U)']
summary_dft

,RF(Smote O&U),XGBc(Smote O&U)
roc_auc(val),0.847,0.845
sensitivity,0.664,0.650
precision,0.212,0.202
F1,0.322,0.309
roc_auc,0.866,0.860


Earlier, Kaggle submission scores:
XGBoost (Oversampling): 0.74243
RandomForest (Undersampling + Oversampling): 0.75238

XGBoost (Undersampling + Oversampling): 0.770

From this, the team is able a better understanding on the potential advantages of combined Oversampling & undersampling technique compared to Oversampling alone.